In [3]:
from bs4 import BeautifulSoup
import pandas as pd

In [53]:
# web scrape brands, names, prices, popularity, and rating of moisturizers from the Sephora website using BeautifulSoup

with open('./data/Sephora_Moisturizers.html', encoding = 'utf-8') as f:
    text = f.read()
dom = BeautifulSoup(text, 'html.parser')
boxes = dom.find_all('div', attrs={'class':'css-1qe8tjm'})
brands = []
names = []
prices = []
popularity = []
ratings = []
for box in boxes:
    brands.append(box.find('span', attrs={'class':'css-12z2u5 eanm77i0'}).text)
    names.append(box.find('span', attrs={'class':'ProductTile-name css-h8cc3p eanm77i0'}).text)
    prices.append(box.find('b', attrs={'class':'css-1f35s9q'}).text)
    popularity.append(box.find('div', attrs={'class':'css-1xk97ib'}).text)
    ratings.append(box.find('span',{'aria-label': True}).get('aria-label'))

In [76]:
# keep more expensive price and edit rating 

import re
updated_prices = []
for price in prices:
    match = re.search(r'\$[\d\.]+\s*-\s*\$([\d\.]+)',price)
    if match:
        max_price = match.group(1)
        updated_prices.append(max_price)
    else:
        updated_prices.append(price[1:])
        
updated_ratings = []
for rate in ratings:
    match = re.search(r'([\d.]+)', rate)
    updated_ratings.append(match[0])

In [77]:
# create csv file to enter scraped moisturizer data

import csv
file = open('scrapes_moisturizers.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Name', 'Brand', 'Price', 'Popularity', 'Rating', 'Store', 'Concern 1', 'Concern 2', 'Concern 3'])

72

In [12]:
pip install selenium==4.9.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 4.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: selenium
    Found existing installation: selenium 4.10.0
    Uninstalling selenium-4.10.0:
      Successfully uninstalled selenium-4.10.0
Note: you may need to restart the kernel to use updated packages.


In [31]:
# get skin concerns for each moisturizer using selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import time

driver = webdriver.Chrome()
local_file = '/Users/nanamisantacruz/Downloads/Moisturizer-Recommender/moisturizer-recommender/data/Sephora_Moisturizers.html'
driver.get(f'file://{local_file}')
grid = driver.find_element(By.CLASS_NAME,'css-1322gsb')
moisturizer_elems = grid.find_elements(By.CLASS_NAME,'css-1qe8tjm')
for elem in moisturizer_elems:
    elem.click()
    indiv_html = driver.page_source()
    indiv_soup = BeautifulSoup(indiv_html, 'html.parser')
    highlights = indiv_soup.find('div', attrs={'class':'css-h2sczi eanm77i0'})
    concerns = highlights.find_all('div', attrs={'class':'css-10t9fn0 eanm77i0'}).text
    if concerns:
        print(concerns)
      

TypeError: 'str' object is not callable

In [78]:
skin_concerns = ['fine lines', 'dryness', 'dullness', 'loss of firmness', 'uneven texture', 'redness', 'dark spots']

In [82]:
import random
for name, brand, price, pop, rating in zip(names,brands,updated_prices,popularity, updated_ratings):
    concern1 = random.choice(skin_concerns)
    concern2 = random.choice(skin_concerns)
    while concern2 == concern1:
        concern2 = random.choice(skin_concerns)
    concern3 = random.choice(skin_concerns)
    while concern3 == concern1 or concern3 == concern2:
        concern3 = random.choice(skin_concerns)
    writer.writerow([name, brand, price, pop, rating, 'Sephora', concern1, concern2, concern3])

In [83]:
import pandas as pd
df = pd.read_csv('scrapes_moisturizers.csv')
df

,Name,Brand,Price,Popularity,Rating,Store,Concern 1,Concern 2,Concern 3
0,Lala Retro™ Whipped Refillable Moisturizer wit...,Drunk Elephant,62.0,2.1K,4,Sephora,redness,dryness,loss of firmness
1,Plum Plump Refillable Hyaluronic Acid Moisturizer,Glow Recipe,39.0,1.6K,4.5,Sephora,dryness,dullness,dark spots
2,Benefiance Wrinkle Smoothing Day Cream SPF 23,Shiseido,75.0,210,4.5,Sephora,dryness,fine lines,dullness
3,Protini™ Polypeptide Firming Refillable Moistu...,Drunk Elephant,68.0,6.8K,4,Sephora,redness,dark spots,uneven texture
4,The Dewy Skin Cream Plumping & Hydrating Moist...,Tatcha,85.0,3.3K,4,Sephora,redness,loss of firmness,fine lines
...,...,...,...,...,...,...,...,...,...
120,Black Tea Anti-Aging Moisturizer with Retinol-...,fresh,95.0,1.4K,5 stars,Sephora,uneven texture,dark spots,fine lines
121,Virgin Marula Luxury Face Oil,Drunk Elephant,68.0,1.7K,4.5 stars,Sephora,loss of firmness,redness,dullness
122,Teatreement™ Moisturizer,Dr. Jart+,35.0,624,4 stars,Sephora,uneven texture,dullness,dryness
123,Adaptogen Deep Moisturizing Cream with Ashwaga...,Youth To The People,58.0,2.1K,4.5 stars,Sephora,redness,loss of firmness,dullness


In [23]:
from bs4 import BeautifulSoup

with open('./data/Ulta_Moisturizers.html', encoding = 'utf-8') as f:
    text = f.read()
    
soup = BeautifulSoup(text, 'html.parser')
ulta_boxes = soup.find_all('li', attrs={'class':'ProductListingResults__productCard'})


In [24]:
ulta_names = []
ulta_brands = []
ulta_prices = []
ulta = []

for box in ulta_boxes:
    ulta_names.append(box.find('span', attrs={'class':'ProductCard__product'}).text)
    ulta_brands.append(box.find('span', attrs={'class':'ProductCard__brand'}).text)
    ulta_prices.append(box.find('div', attrs={'class':'ProductPricing'}).text)
    ulta.append(box.find('div', attrs={'class':'ReviewStarsCard'}).text)

In [25]:
import re

updated_ulta_prices = []
for price in ulta_prices:
    if price[:4]=='Sale':
        if len(price)>60:
            m = re.search(r'Sale Price (\$\d+\.\d+) - (\$\d+\.\d+)',price)
            updated_ulta_prices.append(m[2][1:])
        else:
            m = re.search(r'Sale Price (\$\d+\.\d+)\$\d+\.\d+ Original Price (\$\d+\.\d+)\$\d+\.\d+', price)
            updated_ulta_prices.append(m[1][1:])
    else:
        match = re.search(r'\$[\d\.]+\s*-\s*\$([\d\.]+)',price)
        if match:
            max_price = match.group(1)
            updated_ulta_prices.append(max_price)
        else:
            updated_ulta_prices.append(price[1:])

In [26]:
print(len(updated_ulta_prices))

96


In [27]:
import re

ulta_ratings = []

for rate in ulta:
    match = re.search(r'(\d\.\d)', rate)
    if match:
        ulta_ratings.append(match[0])
    else:
        new_rate = rate[0]+'.0'
        ulta_ratings.append(new_rate)

In [28]:
import re

ulta_popularity = []
for rate in ulta:
    match = re.search(r'^([\d.]+) out of 5 stars ; (\d+) reviews\(([\d,]+)\)$', rate)
    ulta_popularity.append(match[3])

In [29]:
import csv
file = open('scrapes_moisturizers.csv', 'a')
writer = csv.writer(file)

In [30]:
for name, brand, price, pop, rate in zip(ulta_names,ulta_brands,updated_ulta_prices,ulta_popularity, ulta_ratings):
    concern1 = random.choice(skin_concerns)
    concern2 = random.choice(skin_concerns)
    while concern2 == concern1:
        concern2 = random.choice(skin_concerns)
    concern3 = random.choice(skin_concerns)
    while concern3 == concern1 or concern3 == concern2:
        concern3 = random.choice(skin_concerns)
    writer.writerow([name, brand, price, pop, rate, 'Ulta', concern1, concern2, concern3])

In [31]:
import pandas as pd
df = pd.read_csv('scrapes_moisturizers.csv')
df

,Name,Brand,Price,Popularity,Rating,Store,Concern 1,Concern 2,Concern 3
0,Lala Retro™ Whipped Refillable Moisturizer wit...,Drunk Elephant,62.00,2.1K,3.862595,Sephora,dullness,loss of firmness,fine lines
1,Plum Plump Refillable Hyaluronic Acid Moisturizer,Glow Recipe,39.00,1.6K,4.887435,Sephora,uneven texture,dryness,redness
2,Benefiance Wrinkle Smoothing Day Cream SPF 23,Shiseido,75.00,210,4.002778,Sephora,loss of firmness,fine lines,dullness
3,Protini™ Polypeptide Firming Refillable Moistu...,Drunk Elephant,68.00,6.8K,3.973982,Sephora,redness,dryness,uneven texture
4,The Dewy Skin Cream Plumping & Hydrating Moist...,Tatcha,85.00,3.3K,4.635265,Sephora,loss of firmness,dryness,uneven texture
...,...,...,...,...,...,...,...,...,...
129,Natural Moisturizing Factors + PhytoCeramides ...,The Ordinary,22.50,741,4.500000,Ulta,uneven texture,dullness,fine lines
130,Pro-Collagen Marine Cream,ELEMIS,138.00,"1,893",4.600000,Ulta,dryness,dark spots,dullness
131,Face Freezie Cooling Hydration Moisturizer + P...,NYX Professional Makeup,20.00,455,4.400000,Ulta,loss of firmness,dark spots,uneven texture
132,Toleriane Dermallegro Ultra Soothing Repair Fa...,La Roche-Posay,31.99,404,4.300000,Ulta,dryness,uneven texture,dullness


In [32]:
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['Rating'] = np.round(df['Rating'], 1)

In [33]:
df

,Name,Brand,Price,Popularity,Rating,Store,Concern 1,Concern 2,Concern 3
0,Lala Retro™ Whipped Refillable Moisturizer wit...,Drunk Elephant,62.00,2.1K,3.9,Sephora,dullness,loss of firmness,fine lines
1,Plum Plump Refillable Hyaluronic Acid Moisturizer,Glow Recipe,39.00,1.6K,4.9,Sephora,uneven texture,dryness,redness
2,Benefiance Wrinkle Smoothing Day Cream SPF 23,Shiseido,75.00,210,4.0,Sephora,loss of firmness,fine lines,dullness
3,Protini™ Polypeptide Firming Refillable Moistu...,Drunk Elephant,68.00,6.8K,4.0,Sephora,redness,dryness,uneven texture
4,The Dewy Skin Cream Plumping & Hydrating Moist...,Tatcha,85.00,3.3K,4.6,Sephora,loss of firmness,dryness,uneven texture
...,...,...,...,...,...,...,...,...,...
129,Natural Moisturizing Factors + PhytoCeramides ...,The Ordinary,22.50,741,4.5,Ulta,uneven texture,dullness,fine lines
130,Pro-Collagen Marine Cream,ELEMIS,138.00,"1,893",4.6,Ulta,dryness,dark spots,dullness
131,Face Freezie Cooling Hydration Moisturizer + P...,NYX Professional Makeup,20.00,455,4.4,Ulta,loss of firmness,dark spots,uneven texture
132,Toleriane Dermallegro Ultra Soothing Repair Fa...,La Roche-Posay,31.99,404,4.3,Ulta,dryness,uneven texture,dullness


In [34]:
df = pd.read_csv('scrapes_moisturizers.csv')

In [35]:
df

,Name,Brand,Price,Popularity,Rating,Store,Concern 1,Concern 2,Concern 3
0,Lala Retro™ Whipped Refillable Moisturizer wit...,Drunk Elephant,62.00,2.1K,3.862595,Sephora,dullness,loss of firmness,fine lines
1,Plum Plump Refillable Hyaluronic Acid Moisturizer,Glow Recipe,39.00,1.6K,4.887435,Sephora,uneven texture,dryness,redness
2,Benefiance Wrinkle Smoothing Day Cream SPF 23,Shiseido,75.00,210,4.002778,Sephora,loss of firmness,fine lines,dullness
3,Protini™ Polypeptide Firming Refillable Moistu...,Drunk Elephant,68.00,6.8K,3.973982,Sephora,redness,dryness,uneven texture
4,The Dewy Skin Cream Plumping & Hydrating Moist...,Tatcha,85.00,3.3K,4.635265,Sephora,loss of firmness,dryness,uneven texture
...,...,...,...,...,...,...,...,...,...
129,Natural Moisturizing Factors + PhytoCeramides ...,The Ordinary,22.50,741,4.500000,Ulta,uneven texture,dullness,fine lines
130,Pro-Collagen Marine Cream,ELEMIS,138.00,"1,893",4.600000,Ulta,dryness,dark spots,dullness
131,Face Freezie Cooling Hydration Moisturizer + P...,NYX Professional Makeup,20.00,455,4.400000,Ulta,loss of firmness,dark spots,uneven texture
132,Toleriane Dermallegro Ultra Soothing Repair Fa...,La Roche-Posay,31.99,404,4.300000,Ulta,dryness,uneven texture,dullness


In [36]:
def alter_popularity(pop):
    if 'K' in pop:
        return int(float(pop.replace('K',''))*1000)
    if ',' in pop:
        return int(float(pop.replace(',','')))
    else:
        return int(pop)

In [37]:
df['Popularity'] = df['Popularity'].apply(alter_popularity)

In [38]:
import numpy as np

def round_rating(a):
    return np.round(a, 1)

df['Rating'] = df['Rating'].map(round_rating)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        134 non-null    object 
 1   Brand       134 non-null    object 
 2   Price       134 non-null    float64
 3   Popularity  134 non-null    int64  
 4   Rating      134 non-null    float64
 5   Store       134 non-null    object 
 6   Concern 1   134 non-null    object 
 7   Concern 2   134 non-null    object 
 8   Concern 3   134 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 9.5+ KB


In [40]:
df['Name'].value_counts()

Confidence in a Cream Anti-Aging Hydrating Moisturizer           2
Ultra Repair Face Moisturizer                                    2
Intensive Moisture Balance Moisturizer                           1
SA Cream                                                         1
Active Moist Oil-Free Moisturizer                                1
                                                                ..
Daily Greens Oil-Free Gel Moisturizer with Moringa and Papaya    1
The True Cream Moisturizing Bomb With Oak Husk and Vitamin B     1
Ultra Repair Oil-Control Moisturizer                             1
Squalane + Omega Repair Deep Hydration Moisturizer               1
Confidence In A Gel Lotion Oil-Free Moisturizer                  1
Name: Name, Length: 132, dtype: int64

In [41]:
df_unique = df.drop_duplicates(subset='Name', keep='first')

In [42]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132 entries, 0 to 133
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        132 non-null    object 
 1   Brand       132 non-null    object 
 2   Price       132 non-null    float64
 3   Popularity  132 non-null    int64  
 4   Rating      132 non-null    float64
 5   Store       132 non-null    object 
 6   Concern 1   132 non-null    object 
 7   Concern 2   132 non-null    object 
 8   Concern 3   132 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 10.3+ KB


In [43]:
df_unique['Name'].value_counts()

Lala Retro™ Whipped Refillable Moisturizer with Ceramides       1
Dramatically Different Face Moisturizing Gel                    1
SA Cream                                                        1
Active Moist Oil-Free Moisturizer                               1
Clinique Smart Clinical Repair Wrinkle Correcting Face Cream    1
                                                               ..
Ultra Repair Oil-Control Moisturizer                            1
Squalane + Omega Repair Deep Hydration Moisturizer              1
Ultra Repair Face Moisturizer                                   1
Strength Trainer Peptide Boost Firming Moisturizer              1
Confidence In A Gel Lotion Oil-Free Moisturizer                 1
Name: Name, Length: 132, dtype: int64

In [44]:
import csv
file = open('moisturizer_data.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Name', 'Brand', 'Price', 'Popularity', 'Rating', 'Store', 'Concern 1', 'Concern 2', 'Concern 3'])

72

In [45]:
file_path = 'moisturizer_data.csv'
df_unique.to_csv(file_path, index=False)

https://www.pythonguis.com/tutorials/create-gui-tkinter/
https://www.tutorialspoint.com/how-can-i-create-a-dropdown-menu-from-a-list-in-tkinter#:~:text=Let%20us%20suppose%20we%20want,value%20of%20the%20dropdown%20menu.


In [46]:
import pandas as pd
df = pd.read_csv('moisturizer_data.csv')
minPrice = df['Price'].min()
maxPrice = df['Price'].max()
print(minPrice, maxPrice)

7.99 260.0


In [47]:
from tkinter import *
root = Tk()
root.title('Moisturizer Recommender')
root.minsize(400,400)
root.maxsize(800,800)
root.geometry('600x600+50+50')

# GET SKIN CONCERNS
def getSelectedConcerns():
    selected_concerns=[]
    for concern, var in concern_dict.items():
        if var.get()==1:
            selected_concerns.append(concern)
    print(selected_concerns)

skin_concerns = ['fine lines', 'dryness', 'dullness', 'loss of firmness', 'uneven texture', 'redness', 'dark spots']
concern_dict={}
for concern in skin_concerns:
    concern_dict[concern] = IntVar()

Label(root, text='Select Skin Concerns (up to 3)').grid(row=0, column=0, columnspan=2, padx=10, pady=5)

rowNum = 1
for concern in skin_concerns:
    Checkbutton(root, text=concern, variable=concern_dict[concern]).grid(row=rowNum, column=1, sticky=W, padx=10, pady=5)
    rowNum+=1


# GET BUDGET BY CONNECTING TO POSTGRES WITH PSYCOPG2
import psycopg2

conn = psycopg2.connect(host='localhost', database='moisturizer_data', user='postgres', password='Emscagg2003.')
cursor = conn.cursor()
cursor.execute('SELECT MIN(price) AS min_price, MAX(price) AS max_price FROM moisturizers;')
result = cursor.fetchone()
cursor.close()
conn.close()
minPrice, maxPrice = result[0], result[1]

Label(root, text='Select the budget range').grid(row=rowNum, column=1, columnspan=2, padx=5, pady=2)
rowNum+=1
minBudget = Scale(root, from_=minPrice, to=maxPrice, orient=HORIZONTAL, label='Minimum', resolution=0.05)
minBudget.set(minPrice)
minBudget.grid(row=rowNum, column=0, columnspan=2, padx=5, pady=2)
maxBudget = Scale(root, from_=minPrice, to=maxPrice, orient=HORIZONTAL, label='Maximum', resolution=0.05)
maxBudget.set(maxPrice)
maxBudget.grid(row=rowNum, column=3, columnspan=2, padx=5, pady=2)


# GET NUMBER OF RECOMMENDATIONS YOU WANT
menu = StringVar()
menu.set('Select the number of recommendations you want')
rowNum+=1
drop = OptionMenu(root, menu, '1', '3', '5', '10').grid(row=rowNum, column=1, columnspan=2, padx=10, pady=5)

rowNum+=1
Button(root, text='Get Recommendations', command=getSelectedConcerns).grid(row=rowNum, column=1, columnspan=2, padx=10, pady=5)

root.mainloop()